In [1]:
from sklearn import metrics
import processing
import time
import os
from tqdm import tqdm
from torch.optim import Adam
from torch import nn
import numpy as np
import sys
sys.path.append("D:/Experiment")
import torch
from MyKu import processing
from MyKu import MyXLM_TW
import pandas as pd

In [2]:
# 训练准备阶段，设置超参数和全局变量

batch_size = 32
num_epoch = 20  # 训练轮次
check_step = 1  # 用以训练中途对模型进行检验：每check_step个epoch进行一次测试和保存模型

learning_rate = 1e-5  # 优化器的学习率

# 获取训练、测试数据、分类类别总数
en_train_data, en_test_data, de_train_data, de_test_data, hi_train_data, hi_test_data = processing.load_hasoc2020()
categories = 2

en_train_iter, en_test_iter = MyXLM_TW.load_xlm_data(en_train_data, en_test_data, batch_size)
de_train_iter, de_test_iter = MyXLM_TW.load_xlm_data(de_train_data, de_test_data, batch_size)
hi_train_iter, hi_test_iter = MyXLM_TW.load_xlm_data(hi_train_data, hi_test_data, batch_size)
#固定写法，可以牢记，cuda代表Gpu
# torch.cuda.is_available()可以查看当前Gpu是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练模型，因为这里是英文数据集，需要用在英文上的预训练模型：bert-base-uncased
# uncased指该预训练模型对应的词表不区分字母的大小写
# 详情可了解：https://huggingface.co/bert-base-uncased
pretrained_model_name = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
# 创建模型 BertSST2Model
model = MyXLM_TW.MyXlmModel(categories, pretrained_model_name)
# 固定写法，将模型加载到device上，
# 如果是GPU上运行，此时可以观察到GPU的显存增加
model.to(device)

# 训练过程
# Adam是最近较为常用的优化器，详情可查看：https://www.jianshu.com/p/aebcaf8af76e
optimizer = Adam(model.parameters(), learning_rate)  # 使用Adam优化器
loss = nn.CrossEntropyLoss()  # 使用crossentropy作为二分类任务的损失函数

# 记录当前训练时间，用以记录日志和存储
timestamp = time.strftime("%m_%d_%H_%M", time.localtime())

In [4]:

file_name = 'readme_tw.md'
model_save_path = 'D:/Experiment_models_save/twitter-xlm-roberta-base-sentiment/hasoc2020/'
en_temp_best = 0.88
de_temp_best = 0.85
hi_temp_best = 0.6
for epoch in range(1, num_epoch + 1):
    MyXLM_TW.train(model, en_train_iter, device, optimizer, loss, epoch)
    MyXLM_TW.train(model, de_train_iter, device, optimizer, loss, epoch)
    MyXLM_TW.train(model, hi_train_iter, device, optimizer, loss, epoch)
    en_acc_score = MyXLM_TW.test(model, en_test_iter, device, epoch, file_name)
    de_acc_score = MyXLM_TW.test(model, de_test_iter, device, epoch, file_name)
    hi_acc_score = MyXLM_TW.test(model, hi_test_iter, device, epoch, file_name)
    print('\n\n')
    if en_acc_score > en_temp_best and de_acc_score > de_temp_best and hi_acc_score > hi_temp_best:
        en_temp_best, de_temp_best, hi_temp_best = en_acc_score, de_acc_score, hi_acc_score
        MyXLM_TW.save_pretrained(model, model_save_path)
        print(
            f'best en_acc_socre : {en_acc_score}, best de_acc_score : {de_acc_score}, best hi_acc_score : {hi_acc_score}')
print(
    f'best en_acc_socre : {en_temp_best}, best de_acc_score : {de_temp_best}, best hi_acc_score : {hi_acc_score}')


Testing: 100%|██████████| 1592/1592 [00:18<00:00, 83.85it/s]


[[709  76]
 [ 80 727]]
              precision    recall  f1-score   support

           0       0.90      0.90      0.90       785
           1       0.91      0.90      0.90       807

    accuracy                           0.90      1592
   macro avg       0.90      0.90      0.90      1592
weighted avg       0.90      0.90      0.90      1592

Acc : 0.9020100502512562	 F1: 0.9019975218417293


Testing: 100%|██████████| 1027/1027 [00:12<00:00, 81.24it/s]


[[643  84]
 [ 63 237]]
              precision    recall  f1-score   support

           0       0.91      0.88      0.90       727
           1       0.74      0.79      0.76       300

    accuracy                           0.86      1027
   macro avg       0.82      0.84      0.83      1027
weighted avg       0.86      0.86      0.86      1027

Acc : 0.8568646543330087	 F1: 0.8303515141708048


Testing: 100%|██████████| 1269/1269 [00:16<00:00, 78.30it/s]


...

[[737  48]
 [105 702]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       785
           1       0.94      0.87      0.90       807

    accuracy                           0.90      1592
   macro avg       0.91      0.90      0.90      1592
weighted avg       0.91      0.90      0.90      1592

Acc : 0.9038944723618091	 F1: 0.9038479985504724


Testing: 100%|██████████| 1027/1027 [00:10<00:00, 93.77it/s]


[[701  26]
 [117 183]]
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       727
           1       0.88      0.61      0.72       300

    accuracy                           0.86      1027
   macro avg       0.87      0.79      0.81      1027
weighted avg       0.86      0.86      0.85      1027

Acc : 0.8607594936708861	 F1: 0.813250170077759


Testing: 100%|██████████| 1269/1269 [00:14<00:00, 88.01it/s]

[[794 108]
 [244 123]]
              precision    recall  f1-score   support

           0       0.76      0.88      0.82       902
           1       0.53      0.34      0.41       367

    accuracy                           0.72      1269
   macro avg       0.65      0.61      0.61      1269
weighted avg       0.70      0.72      0.70      1269

Acc : 0.7226162332545312	 F1: 0.6149639692445609



best en_acc_socre : 0.9051507537688442, best de_acc_score : 0.8685491723466408, best hi_acc_score : 0.7226162332545312


In [ ]:
from transformers import XLMRobertaForSequenceClassification

xlm = XLMRobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-xlm-roberta-base-sentiment', return_dict=True)
xlm.classifier.out_proj = nn.Linear(768, 2)
print(xlm.eval())